In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method



"""


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

INFO - 2021-01-25 22:35:27,204 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-25 22:35:27,205 - settings - Setting database.user to celiib
INFO - 2021-01-25 22:35:27,206 - settings - Setting database.password to newceliipass
INFO - 2021-01-25 22:35:27,209 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-01-25 22:35:27,210 - settings - Setting enable_python_native_blobs to True
INFO - 2021-01-25 22:35:27,223 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-01-25 22:35:27,477 - settings - Setting enable_python_native_blobs to True


In [3]:
#so that it will have the adapter defined
from datajoint_utils import *

In [4]:
test_mode = True

# Debugging the contains method

In [5]:
import system_utils as su

In [6]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-01-25 22:35:27,662 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-01-25 22:35:27,663 - settings - Setting database.user to celiib
INFO - 2021-01-25 22:35:27,664 - settings - Setting database.password to newceliipass
INFO - 2021-01-25 22:35:27,665 - settings - Setting enable_python_native_blobs to True


Sleeping 88 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-01-25 22:35:27,910 - settings - Setting enable_python_native_blobs to True


# Defining the Table

In [7]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [8]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 5426


In [9]:
#minnie.CellTypes.drop()

In [10]:
import numpy as np
import time
import classification_utils as clu

cell_type_version = 0

verbose = True

@schema
class CellTypes(dj.Computed):
    definition="""
    -> minnie.Decomposition()
    split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    cell_type_version    : tinyint unsigned             # the version of code used for this cell typing classification
    ---
    cell_type_predicted: enum('excitatory','inhibitory','other','unknown') # morphology predicted by classifier
    spine_category: enum('no_spined','sparsely_spined','densely_spined')
    axon_angle_maximum=NULL:double #the anlge of an identified axon
    n_axons: tinyint unsigned             # Number of axon candidates identified
    n_apicals: tinyint unsigned             # Number of apicals identified
    
    spine_density:double              # the number of spines divided by skeletal length for branches analyzed in classification
    n_branches_processed: int unsigned                 # the number branches used for the spine density analysis
    skeletal_length_processed: double                 # The total skeletal length of the viable branches used for the spine density analysis
    n_branches_in_search_radius: int unsigned                 # the number branches existing in the search radius used for spine density
    skeletal_length_in_search_radius : double         # The total skeletal length of the branches existing in the search radius used for spine density
    
    spine_data : longblob     #stores the newly computes spines that were used for the classification
    
    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """
                             
    
    classified_table = du.get_exc_inh_classified_table()
    
    key_source = (minnie.Decomposition & 
                (minnie.NeuronSplitSuggestions.proj()) & 
                (classified_table.proj()))
    
    

    def make(self,key):
        """
        Pseudocode: 
        0) Download the possible neurons from either Decomposition or DecompositionSplit
        
        For Each Neuron
        1) Extract the Segment_id
        3) Run the inhibitory and excitatory classifier
        4) Extract out the maximum axon angle and the spining percentage
        5) Write to the table

        """
        
        whole_pass_time = time.time()
        
        
        #0) Download the possible neurons from either Decomposition or DecompositionSplit
        
        split_table_neuron = (minnie.DecompositionSplit & key)
        if len(split_table_neuron)>0:

            if verbose:
                print("Pulling down neurons from DecompositionSplit")

            neuron_objs,split_indexes = split_table_neuron.fetch("decomposition","split_index")
        else:

            if verbose:
                print("Pulling down neurons from Decomposition")

            neuron_objs = (minnie.Decomposition() & key).fetch("decomposition")
            split_indexes = [0]

            if len(neuron_objs) > 1:
                raise Exception("There were more than 1 neuron when pulling down from Decomposition")
                
        if verbose:
            print(f"Number of Neurons found ={len(neuron_objs)}")

            
            
            
            
        new_keys = []
        for neuron_obj,split_index in zip(neuron_objs,split_indexes):     
        
            #1) Extract the Segment_id
            print(f"\n\n\n---- Working on Neuron {neuron_obj.segment_id}_{neuron_obj.description} ----")
            
            neuron_obj.calculate_spines()
            spine_data = neuron_obj.get_computed_attribute_data(attributes=["spines","spines_volume"])
        
            #3) Run the Classifier:
            (inh_exc_class,
             spine_category,
             axon_angles,
             n_axons,
             n_apicals,
             neuron_spine_density,
             n_branches_processed,
             skeletal_length_processed,
             n_branches_in_search_radius,
             skeletal_length_in_search_radius
             ) = clu.inhibitory_excitatory_classifier(neuron_obj,
                                                return_spine_classification=True,
                                                return_axon_angles=True,
                                                 return_n_axons=True,
                                                 return_n_apicals=True,
                                                 return_spine_statistics=True)
            
            #4) Get the minimum of the axon angles:
            all_axon_angles = []
            for limb_idx,limb_data in axon_angles.items():
                for candidate_idx,cand_angle in limb_data.items():
                    all_axon_angles.append(cand_angle)
                    
            if len(axon_angles)>0:
                axon_angle_maximum = np.max(all_axon_angles)
            else:
                axon_angle_maximum = 0
    
            
        
            
            
            #7) Pass stats and file location to insert
            new_key = dict(key,
                           split_index = split_index,
                           cell_type_version =cell_type_version,
                           
                           cell_type_predicted=inh_exc_class,
                           spine_category=spine_category,
                            axon_angle_maximum=axon_angle_maximum,
                           
                           n_axons=n_axons,
                           n_apicals=n_apicals,
                           
                           spine_density=neuron_spine_density,
                           n_branches_processed=n_branches_processed,
                           skeletal_length_processed=skeletal_length_processed,
                           n_branches_in_search_radius=n_branches_in_search_radius,
                           skeletal_length_in_search_radius=skeletal_length_in_search_radius,
                           
                           spine_data = spine_data,
                           
                           run_time=np.round(time.time() - whole_pass_time,4)
                          )



            new_keys.append(new_key)

        
        self.insert(new_keys, allow_direct_insert=True, skip_duplicates=True)

        print(f"\n\n ------ Total time for {key['segment_id']} = {time.time() - whole_pass_time} ------")
    

# Running the Populate

In [11]:
curr_table = (minnie.schema.jobs & "table_name='__cell_types'")
(curr_table)#.delete()# & "status='error'")
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

table_name className of the table,key_hash key hash,"status if tuple is missing, the job is available",key structure containing the key,error_message error message returned if failed,error_stack error stack if failed,user database user,host system hostname,pid system process id,connection_id connection_id(),timestamp automatic timestamp
__cell_types,01802593d656363cddb5c95feb64e7ef,reserved,=BLOB=,,=BLOB=,celiib@10.21.18.111,71caefafdeec,22362,77420,2021-01-25 16:35:22


In [12]:
import time
pru = reload(pru)
nru = reload(nru)
import neuron_searching as ns
ns = reload(ns)
clu = reload(clu)

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    CellTypes.populate(reserve_jobs=True, suppress_errors=False)
else:
    CellTypes.populate(reserve_jobs=True, suppress_errors=False)
print('Populate Done')

print(f"Total time for CellTypes populate = {time.time() - start_time}")

INFO - 2021-01-25 22:35:36,746 - autopopulate - Found 167 keys to populate
INFO - 2021-01-25 22:35:36,755 - connection - Transaction started
INFO - 2021-01-25 22:35:36,756 - autopopulate - Populating: {'segment_id': 864691136194314572, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'ver': Decimal('30.00'), 'process_version': 3, 'index': 0}


Populate Started
Pulling down neurons from DecompositionSplit
Decompressing Neuron in minimal output mode...please wait
Number of Neurons found =1



---- Working on Neuron 864691136194314572_0_25_soma_0_split ----
query = median_mesh_center > 115 and n_faces_branch>100
smoothness_threshold = 0.12
The median_mesh_center was requested and HAS already been calculated
segment_name before cgal = L0_0



segment_name before cgal = L0_3



segment_name before cgal = L0_4



segment_name before cgal = L0_6


...


segment_name before cgal = L0_15



segment_name before cgal = L0_16



segment_name before cgal = L0_17



segment_name before cgal = L0_18
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-12-bc77d7a4f931>", line 15, in <module>
    CellTypes.populate(reserve_jobs=True, suppress_errors=False)
  File "/usr/local/lib/python3.6/dist-packages/datajoint/autopopulate.py", line 159, in populate
    make(dict(key))
  File "<ipython-input-10-1392787f5cbb>", line 93, in make
    neuron_obj.calculate_spines()
  File "/meshAfterParty/neuron.py", line 2648, in calculate_spines
    shaft_threshold=shaft_threshold)
  File "/meshAfterParty/spine_utils.py", line 419, in get_spine_meshes_unfiltered
    shaft_threshold=shaft_threshold)
  File "/meshAfterParty/spine_utils.py", line 282, in get_spine_meshes_unfiltered_from_mesh
    shaft_threshold = shaft_threshold)
  File "/meshAfterParty/spine_utils.py", line 176, in split_mesh_into_spines_sha

TypeError: object of type 'NoneType' has no len()